# W&B chart expressions demo: Creating Derived Metrics in Charts

Expressions allow you to perform mathematical operations on logged metrics to create new derived values. This notebook demonstrates some ways to use expressions to:
- Convert one metric to another, such as by converting accuracy to error rate.
- Calculate the ratios of two metrics.
- Scale values for better visualization.

In [1]:
# Install W&B
!pip install wandb -q

In [2]:
import wandb
import numpy as np
import random
import time

# Initialize W&B
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: matt-linville (wandb) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Example 1: Train a simple model

Let's simulate a training run and log some metrics, so that we can create some charts that use expressions.

In [3]:
# Start a new run
run = wandb.init(
    project="expression-demo",
    name="training-example",
    config={
        "learning_rate": 0.001,
        "batch_size": 32,
        "epochs": 50
    }
)

# Simulate training metrics
epochs = 50
for epoch in range(epochs):
    # Simulate improving accuracy
    accuracy = 0.5 + (0.4 * epoch / epochs) + random.uniform(-0.02, 0.02)
    accuracy = min(accuracy, 0.98)

    # Simulate decreasing loss
    loss = 2.0 * np.exp(-epoch / 10) + random.uniform(-0.05, 0.05)

    # Simulate validation metrics (slightly worse than training)
    val_accuracy = accuracy - random.uniform(0.02, 0.05)
    val_loss = loss + random.uniform(0.05, 0.15)

    # Simulate learning rate decay
    learning_rate = 0.001 * (0.95 ** (epoch // 10))

    # Simulate memory usage
    memory_used = 1000 + epoch * 20 + random.uniform(-50, 50)
    memory_total = 8000

    # Log metrics
    wandb.log({
        "epoch": epoch,
        "accuracy": accuracy,
        "loss": loss,
        "val_accuracy": val_accuracy,
        "val_loss": val_loss,
        "learning_rate": learning_rate,
        "memory_used": memory_used,
        "memory_total": memory_total
    })

    time.sleep(0.1)  # Small delay to simulate training time

print(f"Training complete! View your run at: {run.url}")
wandb.finish()

Training complete! View your run at: https://wandb.ai/wandb/expression-demo/runs/fjg1cqde


accuracy,▁▁▁▂▂▂▂▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▆▅▅▆▆▆▇▆▇▇▇██▇█
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
learning_rate,███████▆▆▆▆▆▆▆▆▄▄▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁
loss,█▇▇▆▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
memory_total,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
memory_used,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▆▆▅▆▆▆▆▇▇▇▇▇███
val_accuracy,▁▁▂▂▂▂▂▂▂▂▂▂▃▄▄▄▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
val_loss,█▇▇▆▆▅▄▄▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.89661
epoch,49
learning_rate,0.00081


## How to Use Expressions in W&B

After running the cell above, click on the run URL to open your W&B dashboard. Then:

1. **Create a Line Plot Panel**: Click "Add Panel" → "Line Plot"
2. **Select a Metric**: Choose one metric (e.g., `accuracy`)
3. **Add an Expression**: In the panel settings, find the "Expression" field
4. **Try These Examples**:

### Example Expressions to Try:

#### 1. Error Rate
```
1 - accuracy
```
This shows how error decreases as accuracy improves.


#### 2. Percentage Accuracy
```
accuracy * 100
```
Displays accuracy as a percentage (0-100 instead of 0-1).

#### 3. Overfitting Indicator
```
loss / val_loss
```
Values > 1 suggest overfitting (training loss lower than validation).

#### 4. Learning Rate (Scaled)
```
learning_rate * 1000
```
Makes the small learning rate values more visible.

#### 5. Memory Usage Percentage
```
memory_used / memory_total * 100
```
Shows memory usage as a percentage.

## Example 2: Classification Metrics

Let's create another run with precision and recall to demonstrate F1 score calculation.

In [4]:
# Start a new run for classification metrics
run = wandb.init(
    project="expression-demo",
    name="classification-example",
    config={
        "model": "binary_classifier",
        "threshold": 0.5
    }
)

# Simulate classification metrics
steps = 100
for step in range(steps):
    # Simulate improving precision and recall
    t = step / steps
    precision = 0.6 + 0.3 * t + random.uniform(-0.02, 0.02)
    recall = 0.5 + 0.4 * t + random.uniform(-0.02, 0.02)

    # Ensure values stay in valid range
    precision = min(max(precision, 0), 1)
    recall = min(max(recall, 0), 1)

    # Log metrics
    wandb.log({
        "step": step,
        "precision": precision,
        "recall": recall
    })

    time.sleep(0.05)

print(f"Classification run complete! View at: {run.url}")
wandb.finish()

Classification run complete! View at: https://wandb.ai/wandb/expression-demo/runs/795xoboi


precision,▁▁▁▂▁▂▂▂▂▃▃▃▃▃▃▃▄▃▄▄▄▄▄▅▆▆▆▇▇▇▇▇▇▇▇█████
recall,▂▁▁▂▁▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇██
precision,0.88368
recall,0.87713
step,99


### F1 Score Expression

For the classification run above, you can calculate F1 score using:

```
2 * (precision * recall) / (precision + recall)
```

This gives you the harmonic mean of precision and recall.

## Next Steps

Try modifying the code above to:
- Log additional metrics (e.g., `true_positives`, `false_positives`).
- Create custom ratios relevant to your use case.
- Experiment with different mathematical transformations.

Happy experimenting with W&B expressions! 🚀